In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile
import math

from datetime import datetime
%load_ext tensorboard

In [2]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def names_at_batch(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        return x_names, y_names

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])

        # open x image names, resize, normalise and make a numpy array
        x1 = np.asarray([preprocess_input(img_to_array(load_img(file_name[0], target_size=(299, 299)))) for file_name in x_names])
        x2 = np.asarray([preprocess_input(img_to_array(load_img(file_name[1], target_size=(299, 299)))) for file_name in x_names])
        x3 = np.asarray([preprocess_input(img_to_array(load_img(file_name[2], target_size=(299, 299)))) for file_name in x_names])
        # x1 = np.asarray([img_to_array(load_img(file_name[0], target_size=(299, 299))) for file_name in x_names]) / 255.0
        # x2 = np.asarray([img_to_array(load_img(file_name[1], target_size=(299, 299))) for file_name in x_names]) / 255.0
        # x3 = np.asarray([img_to_array(load_img(file_name[2], target_size=(299, 299))) for file_name in x_names]) / 255.0

        return [x1, x2, x3], y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [3]:
def get_filenames_from_dir(directory):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        triplet = []
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            triplet.append(("{0}/{1}/{2}".format(directory, category, file), category_count))
            if len(triplet) == 3:
                x.append([img[0] for img in triplet])
                y.append(triplet[0][1])
                triplet = []
        category_count += 1

    return x, y

In [4]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

1035167: 0
1035185: 1
1035194: 2
1035195: 3
1035204: 4
1035208: 5
1035231: 6
1035290: 7
1035366: 8
1035434: 9
1035542: 10
1035551: 11
1035578: 12
1035864: 13
1035929: 14
1035931: 15
1036066: 16
1036128: 17
1036154: 18
1036192: 19
1036203: 20
1036216: 21
1036255: 22
1036286: 23
1036789: 24
1036796: 25
1036893: 26
1036899: 27
1036917: 28
1036958: 29
1037293: 30
1037319: 31
1037633: 32
4308786: 33
4308787: 34
4308789: 35
4308790: 36
4308800: 37
4308801: 38
4308804: 39
4308805: 40
4308806: 41
4308807: 42
4308811: 43
4308812: 44
4308815: 45
4470539: 46
4470555: 47
4470765: 48
4470801: 49
4471071: 50
4471113: 51
4471202: 52
4471235: 53
4471238: 54
4471269: 55
4472828: 56
4472849: 57
4472858: 58
4472884: 59
4472897: 60
4472900: 61
4472907: 62
4472913: 63
4472929: 64
4473127: 65
4473277: 66
4473297: 67
4473319: 68
4473320: 69
4473617: 70
4473795: 71
4473834: 72
4473860: 73
4473874: 74
4473903: 75
4473974: 76
4474146: 77
4474169: 78
4474653: 79
4474780: 80
4474861: 81
4474974: 82
4474998: 83
44

In [5]:
# make generators with batch size 32 for each set
train_gen = SequenceGenerator(x_train, y_train, 4)
val_gen = SequenceGenerator(x_val, y_val, 4)
test_gen = SequenceGenerator(x_test, y_test, 4)

In [7]:
# Model 1
base_model1 = InceptionV3(weights="imagenet")
for layer in base_model1.layers:
    layer._name += "_1"
    layer.trainable = True

# Model 2
base_model2 = InceptionV3(weights="imagenet")
for layer in base_model2.layers:
    layer._name += "_2"
    layer.trainable = True

# Model 3
base_model3 = InceptionV3(weights="imagenet")
for layer in base_model3.layers:
    layer._name += "_3"
    layer.trainable = True

In [8]:
out_layer = keras.layers.concatenate([base_model1.layers[-2].output, base_model2.layers[-2].output, base_model3.layers[-2].output])
predictions = keras.layers.Dense(291, activation='softmax')(out_layer)

model = keras.Model(inputs=[base_model1.input, base_model2.input, base_model3.input], outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
logdir = "logs/3model_ensemb_unshared_concat" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [11]:
model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback], epochs=20)

Epoch 1/20
3709/3709 [==============================] - 797s 207ms/step - loss: 5.5457 - accuracy: 0.0127 - val_loss: 21.1147 - val_accuracy: 0.0056
Epoch 2/20
3709/3709 [==============================] - 751s 202ms/step - loss: 5.1228 - accuracy: 0.0324 - val_loss: 5.7034 - val_accuracy: 0.0303
Epoch 3/20
3709/3709 [==============================] - 753s 203ms/step - loss: 4.7243 - accuracy: 0.0533 - val_loss: 6.9072 - val_accuracy: 0.0441
Epoch 4/20
3709/3709 [==============================] - 754s 203ms/step - loss: 4.2225 - accuracy: 0.0900 - val_loss: 4.5086 - val_accuracy: 0.0622
Epoch 5/20
3709/3709 [==============================] - 751s 202ms/step - loss: 3.5757 - accuracy: 0.1693 - val_loss: 3.4151 - val_accuracy: 0.1879
Epoch 6/20
3709/3709 [==============================] - 749s 202ms/step - loss: 2.8591 - accuracy: 0.2759 - val_loss: 2.7224 - val_accuracy: 0.3057
Epoch 7/20
3709/3709 [==============================] - 762s 205ms/step - loss: 2.2861 - accuracy: 0.3792 - val

In [12]:
model.evaluate(test_gen)

789/789 [==============================] - 53s 67ms/step - loss: 1.6034 - accuracy: 0.6065


[1.6034029722213745, 0.6065313816070557]